In [ ]:
import os
import sys
import numpy as np
import pandas as pd

total_df = pd.DataFrame()

for test_dataset in ['TCGA']:
    for model in ['WANCDR', 'WANCDR_5Critic', 'WANCDR_Gaussian', 'WANCDR_Gaussian_5Critic']:
        for strategy in ['CDRTCGA']:
            for metric in ['W_distance']:
                path = f'./logs/{test_dataset}/{model}/{strategy}/{metric}/TCGA_results.csv'
                if os.path.exists(path):
                    df = pd.read_csv(path)
                    df['mode'] = f'{test_dataset}_{model}_{strategy}_{metric}'
                    total_df = pd.concat([total_df, df], ignore_index=True)
                    
grouped = total_df.groupby([ 'mode', 'nz', 'd_dim', 'lr', 'lr_adv','lam', 'batch_size'])[
    ['Test AUC', 'Train AUC', 'Val AUC','Val W_distance', 'Test W_distance']
].mean()
grouped

,,,,,,,Test AUC,Train AUC,Val AUC,Val W_distance,Test W_distance
mode,nz,d_dim,lr,lr_adv,lam,batch_size,,,,,
TCGA_WANCDR_5Critic_CDRTCGA_W_distance,100,256,0.00001,0.00002,0.0010,"[256, 28]",0.432560,0.627149,0.653295,0.074873,0.077317
TCGA_WANCDR_CDRTCGA_W_distance,100,256,0.00010,0.00010,0.0001,"[128, 14]",0.704466,0.802701,0.799029,0.250099,0.254619
TCGA_WANCDR_Gaussian_5Critic_CDRTCGA_W_distance,100,256,0.00010,0.00010,0.0001,"[128, 14]",0.641598,0.799135,0.774610,0.016936,0.015042
TCGA_WANCDR_Gaussian_CDRTCGA_W_distance,256,128,0.00010,0.00002,0.0001,"[128, 14]",0.631137,0.877178,0.804852,0.017954,-0.003299


In [15]:
import os
import numpy as np
import pandas as pd

# 1) 로그 파일 합치기 (이미 하신 부분)
total_df = pd.DataFrame()
for test_dataset in ['TCGA']:
    for model in ['WANCDR', 'WANCDR_5Critic', 'WANCDR_Gaussian', 'WANCDR_Gaussian_5Critic']:
        for strategy in ['5FoldCV', 'CDRTCGA']:
            for metric in ['AUC', 'Loss', 'W-distance']:
                path = f'./logs/{test_dataset}/{model}/{strategy}/{metric}/TCGA_results.csv'
                if os.path.exists(path):
                    df = pd.read_csv(path)
                    df['mode'] = f'{test_dataset}_{model}_{strategy}_{metric}'
                    total_df = pd.concat([total_df, df], ignore_index=True)

# 2) 원하는 컬럼으로 그룹화하여 평균 계산
grouped = (
    total_df
    .groupby(['mode', 'nz', 'd_dim', 'lr', 'lr_adv', 'lam', 'batch_size'])
    [['Test AUC', 'Train AUC', 'Val AUC', 'Val W_distance', 'Test W_distance']]
    .mean()
    .reset_index()
)

# 3) Val AUC가 가장 높은 그룹
best_val = grouped.loc[grouped['Val AUC'].idxmax()]
print("=== Val AUC 기준 최우수 그룹 ===")
print(best_val)

# 4) Test AUC가 가장 높은 그룹
best_test = grouped.loc[grouped['Test AUC'].idxmax()]
print("\n=== Test AUC 기준 최우수 그룹 ===")
print(best_test)

# 5) (옵션) 보기 좋게 DataFrame 형태로
best_val_df  = best_val.to_frame().T
best_test_df = best_test.to_frame().T

print("\n-- Best by Val AUC --")
print(best_val_df)
print("\n-- Best by Test AUC --")
print(best_test_df)


=== Val AUC 기준 최우수 그룹 ===
mode               TCGA_WANCDR_CDRTCGA_AUC
nz                                     100
d_dim                                  256
lr                                  0.0001
lr_adv                              0.0001
lam                                 0.0001
batch_size                       [128, 14]
Test AUC                            0.6723
Train AUC                          0.82754
Val AUC                           0.821682
Val W_distance                     0.43196
Test W_distance                   0.417404
Name: 1, dtype: object

=== Test AUC 기준 최우수 그룹 ===
mode               TCGA_WANCDR_Gaussian_5Critic_CDRTCGA_AUC
nz                                                      100
d_dim                                                   256
lr                                                   0.0001
lr_adv                                               0.0001
lam                                                  0.0001
batch_size                                     

In [5]:
total_df = pd.DataFrame()
for test_dataset in ['GDSC', 'TCGA']:
    for model in ['WANCDR_Gaussian', 'WANCDR_Gaussian_5Critic']:
        for strategy in ['5FoldCV']:
            for metric in ['AUC', 'Loss', 'W-distance']:
                path = f'./logs/{test_dataset}/{model}/{strategy}/{metric}/results.csv'
                if os.path.exists(path):
                    df = pd.read_csv(path)
                    df['mode'] = f'{test_dataset}_{model}_{strategy}_{metric}'
                    total_df = pd.concat([total_df, df], ignore_index=True)

grouped = total_df.groupby([ 'mode', 'nz', 'd_dim', 'lr', 'lr_adv', 'lam', 'batch_size'])[
    ['Test AUC', 'Train AUC', 'Val AUC','Val W_distance', 'Test W_distance']
].mean()
grouped

Test AUC  \
mode                                     nz  d_dim lr      lr_adv   lam    batch_size             
GDSC_WANCDR_Gaussian_5Critic_5FoldCV_AUC 100 100   0.00001 0.000002 0.0010 [256, 28]   0.584020   
                                                           0.000020 0.0001 [128, 14]   0.588450   
                                                   0.00010 0.000020 0.0001 [256, 28]   0.655368   
                                                                    0.0010 [128, 14]   0.598662   
                                             256   0.00010 0.000020 0.0001 [128, 14]   0.665964   
                                                                    0.0010 [128, 14]   0.641044   
                                                                           [256, 28]   0.574315   
                                         128 100   0.00001 0.000002 0.0010 [256, 28]   0.570286   
                                             128   0.00001 0.000020 0.0010 [256, 28]   0.578769   
                                                   0.00010 0.000002 0.0001 [256, 28]   0.635081   
                                                           0.000020 0.0010 [128, 14]   0.644721   
                                             256   0.00001 0.000020 0.0010 [128, 14]   0.574911   
                                         256 100   0.00001 0.000002 0.0010 [256, 28]   0.545709   
                                             128   0.00001 0.000020 0.0001 [128, 14]   0.563180   
                                                   0.00010 0.000002 0.0010 [256, 28]   0.616435   
                                                           0.000020 0.0001 [128, 14]   0.641533   
                                             256   0.00001 0.000002 0.0001 [256, 28]   0.585948   
                                                                    0.0010 [128, 14]   0.612953   
                                                           0.000020 0.0001 [128, 14]   0.552244   
                                                   0.00010 0.000002 0.0010 [256, 28]   0.607401   
GDSC_WANCDR_Gaussian_5FoldCV_AUC         100 100   0.00001 0.000010 0.0010 [256, 28]   0.524619   
                                                           0.000100 0.0001 [128, 14]   0.524502   
                                                   0.00010 0.000100 0.0001 [256, 28]   0.663708   
                                                                    0.0010 [128, 14]   0.667972   
                                             256   0.00010 0.000100 0.0001 [128, 14]   0.646102   
                                                                    0.0010 [128, 14]   0.656025   
                                                                           [256, 28]   0.665685   
                                         128 100   0.00001 0.000010 0.0010 [256, 28]   0.549347   
                                             128   0.00001 0.000100 0.0010 [256, 28]   0.581195   
                                                   0.00010 0.000010 0.0001 [256, 28]   0.609806   
                                                           0.000100 0.0010 [128, 14]   0.655011   
                                             256   0.00001 0.000100 0.0010 [128, 14]   0.606874   
                                         256 100   0.00001 0.000010 0.0010 [256, 28]   0.565578   
                                             128   0.00001 0.000100 0.0001 [128, 14]   0.609311   
                                                   0.00010 0.000010 0.0010 [256, 28]   0.619812   
                                                           0.000100 0.0001 [128, 14]   0.650548   
                                             256   0.00001 0.000010 0.0001 [256, 28]   0.569275   
                                                                    0.0010 [128, 14]   0.568853   
                                                           0.000100 0.0001 [128, 14]   0.573887   
                            

In [14]:
total_df.columns

Index(['nz', 'd_dim', 'lr', 'lr_adv', 'lam', 'batch_size', 'Epoch',
       'Train AUC', 'Train Accuracy', 'Train F1', 'Train Recall',
       'Train Precision', 'Train W_distance', 'Train critic_loss',
       'Train gen_loss', 'Train total_loss', 'Val AUC', 'Val Accuracy',
       'Val F1', 'Val Recall', 'Val Precision', 'Val W_distance',
       'Val critic_loss', 'Val gen_loss', 'Val total_loss', 'Test AUC',
       'Test Accuracy', 'Test F1', 'Test Recall', 'Test Precision',
       'Test W_distance', 'Test critic_loss', 'Test gen_loss',
       'Test total_loss', 'Iteration', 'mode'],
      dtype='object')

In [7]:
import os
import glob
import imageio

def create_umap_gif(image_dir, gif_save_path, fps=2):
    """
    image_dir: PNG 이미지들이 저장된 폴더 경로
    gif_save_path: 생성할 GIF 저장 경로
    fps: 초당 프레임 수 (예: 2fps = 0.5초/프레임)
    """
    # .png 파일 정렬 (epoch 숫자 기준)
    image_paths = sorted(
        glob.glob(os.path.join(image_dir, "epoch*.png")),
        key=lambda x: int(os.path.basename(x).split("epoch")[1].split(".")[0])
    )

    if not image_paths:
        print("No .png files found in the specified directory.")
        return

    print(f"Found {len(image_paths)} images. Creating GIF at {fps} fps...")

    # 이미지 불러오기
    frames = [imageio.imread(png) for png in image_paths]

    # 저장 (fps 직접 지정)
    imageio.mimsave(gif_save_path, frames, fps=fps)

    print(f"GIF saved to {gif_save_path}")


In [9]:
create_umap_gif(
    image_dir="/workspace/W-PANCDR/src/logs/GDSC/WANCDR_Gaussian/5FoldCV/AUC/umap_result/fold0",
    gif_save_path="/workspace/W-PANCDR/src/logs/GDSC/WANCDR_Gaussian/5FoldCV/AUC/umap_result/fold0_umap.gif",
    fps=3
)


Found 47 images. Creating GIF at 3 fps...


/tmp/ipykernel_2309699/656033718.py:24: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames = [imageio.imread(png) for png in image_paths]


GIF saved to /workspace/W-PANCDR/src/logs/GDSC/WANCDR_Gaussian/5FoldCV/AUC/umap_result/fold0_umap.gif


In [1]:
import pandas as pd
import os

# csv path: /workspace/W-PANCDR/src/logs/TCGA/WANCDR_5Critic/CDRTCGA/AUC/TCGA_results.csv
csv_path = '/workspace/W-PANCDR/src/logs/TCGA/WANCDR_5Critic/CDRTCGA/AUC/TCGA_results.csv'
df = pd.read_csv(csv_path)
df[['Val AUC', 'Test AUC']].describe()

,Val AUC,Test AUC
count,10.000000,10.000000
mean,0.810259,0.698781
std,0.006269,0.020358
min,0.800263,0.668144
25%,0.806500,0.686194
50%,0.810086,0.698404
75%,0.813999,0.714184
max,0.821229,0.727112


In [ ]:
hyperparameter_set = ['nz', 'd_dim', 'lr', 'lr_adv', 'batch_size']

Index(['nz', 'd_dim', 'lr', 'lr_adv', 'lam', 'batch_size', 'Epoch',
       'Train AUC', 'Train Accuracy', 'Train F1', 'Train Recall',
       'Train Precision', 'Train W_distance', 'Train critic_loss',
       'Train gen_loss', 'Train total_loss', 'Val AUC', 'Val Accuracy',
       'Val F1', 'Val Recall', 'Val Precision', 'Val W_distance',
       'Val critic_loss', 'Val gen_loss', 'Val total_loss', 'Test AUC',
       'Test Accuracy', 'Test F1', 'Test Recall', 'Test Precision',
       'Test W_distance', 'Test critic_loss', 'Test gen_loss',
       'Test total_loss', 'Iteration'],
      dtype='object')

In [19]:
# Test_dataset: GDSC/TCGA
# 5FoldCV
# Model : WANCDR/WANCDR_Gaussian
# Strategy: 5FoldCV
# Test Metric: AUC/Loss/W-distance
# ex: /workspace/W-PANCDR/src/logs/GDSC/WANCDR/5FoldCV/AUC/results.csv
import os
import sys
import numpy as np
import pandas as pd

total_df = pd.DataFrame()

for test_dataset in ['TCGA']:
    for model in ['WANCDR', 'WANCDR_5Critic', 'WANCDR_Gaussian', 'WANCDR_Gaussian_5Critic']:
        for strategy in ['CDRTCGA']:
            for metric in ['AUC']:
                path = f'./logs/{test_dataset}/{model}/{strategy}/{metric}/TCGA_results.csv'
                if os.path.exists(path):
                    df = pd.read_csv(path)
                    df['mode'] = f'{test_dataset}_{model}_{strategy}_{metric}'
                    total_df = pd.concat([total_df, df], ignore_index=True)
                    
grouped = total_df.groupby(['mode', 'nz', 'd_dim', 'lr', 'lr_adv','lam', 'batch_size'])[
    ['Test AUC', 'Train AUC', 'Val AUC','Val W_distance', 'Test W_distance']
].mean()
grouped

,,,,,,,Test AUC,Train AUC,Val AUC,Val W_distance,Test W_distance
mode,nz,d_dim,lr,lr_adv,lam,batch_size,,,,,
TCGA_WANCDR_5Critic_CDRTCGA_AUC,100,256,0.00001,0.00002,0.0010,"[256, 28]",0.654119,0.749894,0.751911,0.354861,0.371078
TCGA_WANCDR_CDRTCGA_AUC,100,256,0.00010,0.00010,0.0001,"[128, 14]",0.672300,0.827540,0.821682,0.431960,0.417404
TCGA_WANCDR_Gaussian_5Critic_CDRTCGA_AUC,100,256,0.00010,0.00010,0.0001,"[128, 14]",0.679610,0.799570,0.787683,0.025768,0.023666
TCGA_WANCDR_Gaussian_CDRTCGA_AUC,256,128,0.00010,0.00002,0.0001,"[128, 14]",0.614773,0.844627,0.818434,-0.153821,-0.153846
